In [2]:
import sys
import os
import pandas as pd

# Cargar el archivo CSV
file_path = 'datos_jugadores_biwenger.csv'
df = pd.read_csv(file_path, delimiter=';')

# Mostrar el precio de cada jugador
print(df[['Jugador', 'Precio']])


            Jugador    Precio
0        Bellingham  53000000
1            Dovbyk  24000000
2             Sávio  10000000
3          Williams  34000000
4           Sørloth  30000000
..              ...       ...
430         Fuidias   7000000
431     Pablo Marín   7000000
432    Matías Árbol   7000000
433  Alberto Flores   7000000
434     Adama Boiro   7000000

[435 rows x 2 columns]


In [9]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Cargar los archivos CSV
df_biwenger = pd.read_csv('datos_jugadores_biwenger.csv', delimiter=';')
df_statistics = pd.read_csv('all_players_statistics.csv')

# Función para encontrar el nombre más similar
def find_best_match(name, choices, scorer, threshold=80):
    best_match, score = process.extractOne(name, choices, scorer=scorer)
    return best_match if score >= threshold else None

# Crear un diccionario para almacenar las coincidencias
matches = {}

# Lista de nombres en el archivo de estadísticas
statistics_names = df_statistics['Player Name'].tolist()

# Encontrar las mejores coincidencias para cada nombre en el archivo de biwenger
for player in df_biwenger['Jugador']:
    best_match = find_best_match(player, statistics_names, fuzz.token_set_ratio)
    matches[player] = best_match

# Crear una nueva columna en el dataframe de biwenger con los nombres coincidentes
df_biwenger['Player Name'] = df_biwenger['Jugador'].map(matches)

# Filtrar filas donde no se encontró una coincidencia adecuada
df_biwenger = df_biwenger.dropna(subset=['Player Name'])

# Unir ambos dataframes utilizando la columna 'Player Name'
merged_df = pd.merge(df_biwenger, df_statistics, on='Player Name')

# Eliminar duplicados basados en una columna clave, por ejemplo, 'Player Name'
merged_df = merged_df.drop_duplicates(subset=['Player Name'])

# Eliminar la columna 'Jugador'
merged_df = merged_df.drop(columns=['Jugador'])

# Eliminar la columna 'Team'
merged_df = merged_df.drop(columns=['Team'])

# Reordenar las columnas para que 'Player Name' sea la primera
columns_order = ['Player Name'] + [col for col in merged_df.columns if col != 'Player Name']
merged_df = merged_df[columns_order]

# Guardar el dataframe combinado en un nuevo archivo CSV
output_path = 'players_prize.csv'
merged_df.to_csv(output_path, index=False)

# Mostrar el precio y el nombre del jugador
print(merged_df[['Player Name', 'Precio']])


         Player Name    Precio
0    jude-bellingham  53000000
1     inaki-williams  34000000
2       lamine-yamal   8000000
3               isco  16000000
4         iago-aspas  45000000
..               ...       ...
323     unai-marrero   9000000
324        jose-pozo   8000000
325     thomas-lemar  21000000
326  cristian-rivero   7000000
327  aaron-escandell   8000000

[323 rows x 2 columns]
